In [1]:
import numpy as np
import qutip as qt

In [2]:
# Determine whether eavesdropping will take place
eavesdropper_present = False

In [3]:
# Define converting functions
def message_to_binary_str(message):
    return ''.join(format(ord(i), '08b') for i in message)

def binary_str_to_message(bin_str):
    char_list = []
    for i in range(0, len(bin_str), 8):
        ch = chr(int(bin_str[i:i+8], 2))
        char_list.append(ch)
        
    return ''.join(char_list)    

In [4]:
# Ask for message input
is_ascii = False

while not is_ascii:
    message = str(input("Enter message to be encrypted: "))
    is_ascii = all(ord(c) < 128 for c in message)  # check if message is in ASCII

binary_message =  message_to_binary_str(message)

In [5]:
# Determine message length and the lenth of the random sequances
n = len(binary_message)
m = 6*n

Preparation phase

In [6]:
#  Define the constants that Bob and Alice agree on in the preparation phase
RECTILINEAR_BASIS = 0
DIAGONAL_BASIS = 1

# In rectilinear basis
HORIZONTAL_POL = 0
VERTICAL_POL = 1

# In diagonal basis
DIAGONAL_45_POL = 0
DIAGONAL_135_POL = 1

In [7]:
# Generate Alice's and Bob's random bases sequences of size m
alice_rand_bases_seq = np.random.choice([RECTILINEAR_BASIS, DIAGONAL_BASIS], size=m)
bob_rand_bases_seq = np.random.choice([RECTILINEAR_BASIS, DIAGONAL_BASIS], size=m)

# Generate Alice's random bit sequence of size m
alice_rand_bit_seq = np.random.choice([0, 1], size=m).tolist()

if eavesdropper_present:
    # Generate Eve's random bases sequence of size m
    eve_rand_bases_seq = np.random.choice([RECTILINEAR_BASIS, DIAGONAL_BASIS], size=m)
            

In [8]:
# Describe bases of Hilbert vector space
basis_0 = qt.basis(2,0)
basis_1 = qt.basis(2,1)

# Describe polarization states in Hilbert vector space
photon_h = basis_0                            # horizontally polarized photon
photon_v = basis_1                            # vertically polarized photon
photon_d45 = (basis_0 + basis_1).unit()       # diagonally polarized photon (45 deg)
photon_d135 = ((-1)*basis_0 + basis_1).unit() # diagonally polarized photon (135 deg)

In [9]:
b = qt.Bloch()
b.add_states(photon_h)
b.add_states(photon_v)
b.add_states(photon_d45)


In [10]:
# Define the measurement operators simulating Bob's choice of polarization filters
vertical_filter = qt.Qobj([[0, 0],
                           [0, 1]])        # Bob uses vertically oriented filter for measurement in rectilinear basis

diagonal45_filter = qt.Qobj([[0.5, 0.5],
                             [0.5, 0.5]])  # Bob uses diagonally oriented filter (45 deg) for measurement in diagonal basis

In [11]:
qt.measurement.measure_observable(photon_v, diagonal45_filter)  # example of nondeterministic measurement (rerun the cell)

(0.0,
 Quantum object: dims=[[2], [1]], shape=(2, 1), type='ket', dtype=Dense
 Qobj data =
 [[-0.70710678]
  [ 0.70710678]])

Transmission phase

In [12]:
def pick_photon_polarization(basis, bit_value):
    # Polarization of the photon Alice sends depends on her random sequances
    if basis == RECTILINEAR_BASIS:
        if bit_value == HORIZONTAL_POL:
            photon = photon_h
            sign = "H"
        else:  # bit_value == VERTICAL_POL:
            photon = photon_v
            sign = "V"
            
    else:  # basis == DIAGONAL_BASIS
        if bit_value == DIAGONAL_45_POL:
            photon = photon_d45
            sign = "D45"
        else:  # bit_value == DIAGONAL_135_POL
            photon = photon_d135
            sign = "D135"
            
    return photon, sign

In [13]:
def measure_polarization(photon, basis):
    pol_filter = vertical_filter if basis == RECTILINEAR_BASIS else diagonal45_filter

    passed_filter, photon_out = qt.measurement.measure_observable(photon, pol_filter)
    
    if pol_filter == vertical_filter:
        if passed_filter:
            value = VERTICAL_POL    # if photon passes, it is assumed to have the polarization of the filter
        else:
            value = HORIZONTAL_POL  # if photon doesn't pass, it is assumed to be orthogonal to the filter
            
    else:
        if passed_filter:
            value = DIAGONAL_45_POL
        else:
            value = DIAGONAL_135_POL
            
    return value, photon_out

In [14]:
# Perform transmission

bob_measured_values = []
photons_sent = [] # keep track of the photons Alice sent (for demonstration purposes)

for basis_a, bit_value, basis_b, i in zip(alice_rand_bases_seq, alice_rand_bit_seq, bob_rand_bases_seq, range(m)):
    
    # Alice picks a polarized foton source according to her random sequances
    photon, sign = pick_photon_polarization(basis_a, bit_value)
    photons_sent.append(sign)
            
    # Alice sends the picked photon to Bob
    if eavesdropper_present:
        _, photon = measure_polarization(photon, eve_rand_bases_seq[i])
    
    #Bob measures the photon
    value, _ = measure_polarization(photon, basis_b)
    bob_measured_values.append(int(value))  # append value to the end of Bob's measurements sequence

Elimination phase

In [15]:
# 3) Elimination phase

# Alice and Bob compare their random bases sequances
bases_disagreement_indices = np.where(alice_rand_bases_seq != bob_rand_bases_seq)[0]

In [16]:
# Bob removes elements which he measured in the incorrect base from his measurements sequence
for i in np.flip(bases_disagreement_indices):
    bob_measured_values.pop(i)
    
# ALice removes those elements from her random bit sequence
for i in np.flip(bases_disagreement_indices):
    alice_rand_bit_seq.pop(i)

In [17]:
# 4) Error check phase

# Bob picks random subset of his measured values sequence, 1/3 of the sequence lenth (after elimination phase) long
error_check_indices = np.random.randint(0, len(bob_measured_values), len(bob_measured_values)//3)

In [18]:
error_check_indices  # Bob makes indices public for Alice to pick the same elements from her sequence

array([  8,  39,  16,  94,  11, 100,  21,  93,  59,  85, 113,  29, 115,
        58, 109,  31,  62,   3,  38, 104, 113,  33,  61,  81, 102,  48,
        16, 106,  19,  70,  94, 105,  70,  20,  32,  94, 105,  48,  91])

In [19]:
bob_error_check_subset = []
alice_error_check_subset = []

for i in np.flip(np.sort(error_check_indices)):
    bob_el = bob_measured_values.pop(i)
    bob_error_check_subset.append(bob_el)
    
    alice_el = alice_rand_bit_seq.pop(i)
    alice_error_check_subset.append(alice_el)
    

In [20]:
# Compare chosen subsets
sequences_identical = bob_measured_values == alice_rand_bit_seq

if sequences_identical:
    secret_key = alice_rand_bit_seq[:n]  # use first n bits of final sequance as key
    print("Key was safely established.")
    
else:
    raise SystemExit("Eavesdropper was detected! Key couldn't be safely established.")
    # The bellow code is not executed, communication has to be repeated until a safe key is established.

Key was safely established.


Encryption and decryption

In [25]:
def encrypt_message(message, key_seq):
    key = ''.join(map(str, key_seq))
    bin_message = message_to_binary_str(message)
    
    # Perform binary XOR on the message and the key bitwise
    encrypted_bin_seq = [str(int(m) ^ int(k)) for m, k in zip(bin_message, key)]
    
    encrypted_bin_str = ''.join(encrypted_bin_seq)
    encrypted_message = binary_str_to_message(encrypted_bin_str)
    
    return encrypted_message


def decrypt_message(message, key_seq):
    return encrypt_message(message, key_seq)  # messages are encrypted en decrypted the same way in Vernam binary cipher
    

In [22]:
encrypted_message = encrypt_message(message, secret_key)

print("The encrypted message is: " + encrypted_message)

The encrypted message is: ¯çAÖ


In [23]:
decrypted_message = decrypt_message(encrypted_message, bob_measured_values[:n])

print("The decrypted message is: " + decrypted_message)

The decrypted message is: hello
